<a href="https://colab.research.google.com/github/RafsanJany-44/SIgnal_Processing/blob/main/Signal_Processing_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/talhaanwarch/youtube-tutorials

In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [2]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.8 MB/s eta 0:00:00


In [3]:
from glob import glob
import os
import mne
import pandas
import numpy as np
import matplotlib.pyplot as plt

In [4]:
data_path = "/content/drive/MyDrive/Signal_Data/dataverse_files"

In [5]:
all_file_path = glob(data_path+"/*.edf")
print(len(all_file_path))

28


In [6]:
all_file_path

['/content/drive/MyDrive/Signal_Data/dataverse_files/h12.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h05.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h06.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h04.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h08.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h09.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h11.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h02.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h07.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h03.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h01.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h10.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/s01.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/s07.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h13.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h1

In [7]:
healthy_file_path = []
patient_file_path = []

for i in all_file_path:
  if str(os.path.basename(i))[0] == 'h':
    healthy_file_path.append(i)
  else:
    patient_file_path.append(i)

In [8]:
def read_data(file_path):
  data = mne.io.read_raw_edf(file_path, preload=True)
  data.set_eeg_reference()
  data.filter(l_freq=0.5,h_freq=45)
  epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
  array=epochs.get_data()
  return array

In [9]:

sample_data = read_data(healthy_file_path[0])

Extracting EDF parameters from /content/drive/MyDrive/Signal_Data/dataverse_files/h12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 224999  =      0.000 ...   899.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
224 matching events found
No baseline correction applied
0 projection i

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


In [10]:
sample_data.shape   #epochs,channels, length of signals

(224, 19, 1250)

In [11]:
%%capture
control_epochs_array = []
patient_epochs_array = []

for i in healthy_file_path:
  control_epochs_array.append(read_data(i))

for i in patient_file_path:
  patient_epochs_array.append(read_data(i))

In [12]:
control_epochs_array[0].shape

(224, 19, 1250)

In [13]:
patient_epochs_array[0].shape

(211, 19, 1250)

In [14]:

#labeling control as 0. -> Unhealthy
control_epochs_labels = []
for i in control_epochs_array:
  control_epochs_labels.append(len(i)*[0])


#labeling healthy as 1. -> Unhealthy
patients_epochs_labels = []
for i in patient_epochs_array:
  patients_epochs_labels.append(len(i)*[1])

print(len(control_epochs_labels),len(patients_epochs_labels))

14 14


In [15]:
data_list=control_epochs_array+patient_epochs_array
label_list=control_epochs_labels+patients_epochs_labels

In [16]:
#spliting data as subject, makig group for each subject

group_list = []


for i,j in enumerate(data_list):
  group_list.append([i]*len(j))

len(group_list)




28

In [17]:
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)
group_array = np.hstack(group_list)
data_array.shape,label_array.shape,group_array.shape

((7201, 19, 1250), (7201,), (7201,))

In [24]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(19,1250)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    #model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    #model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    #model.add(AveragePooling1D(pool_size=2,strides=2))#8
    #model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11

    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 17, 5)             18755     
                                                                 
 batch_normalization (Batch  (None, 17, 5)             20        
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 17, 5)             0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 8, 5)              0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 5)              80        
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 6, 5)              0

In [25]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()

In [ ]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=50,batch_size=10,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])

Epoch 1/50
575/575 [==============================] - 18s 10ms/step - loss: 0.7086 - accuracy: 0.5707 - val_loss: 0.7183 - val_accuracy: 0.3693
Epoch 2/50
575/575 [==============================] - 4s 6ms/step - loss: 0.6767 - accuracy: 0.5944 - val_loss: 0.7313 - val_accuracy: 0.3699
Epoch 3/50
575/575 [==============================] - 4s 6ms/step - loss: 0.6704 - accuracy: 0.5931 - val_loss: 0.7455 - val_accuracy: 0.3713
Epoch 4/50
575/575 [==============================] - 4s 8ms/step - loss: 0.6684 - accuracy: 0.5961 - val_loss: 0.7427 - val_accuracy: 0.3686
Epoch 5/50
575/575 [==============================] - 4s 6ms/step - loss: 0.6656 - accuracy: 0.5963 - val_loss: 0.7444 - val_accuracy: 0.3693
Epoch 6/50
575/575 [==============================] - 4s 7ms/step - loss: 0.6647 - accuracy: 0.5949 - val_loss: 0.7373 - val_accuracy: 0.3699
Epoch 7/50
575/575 [==============================] - 4s 7ms/step - loss: 0.6597 - accuracy: 0.5938 - val_loss: 0.7296 - val_accuracy: 0.3720
Epoc

In [ ]:
np.mean(accuracy)